In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
from sklearn import linear_model

from sklearn.feature_selection import RFE
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import make_scorer, r2_score, mean_squared_error, auc, mean_absolute_error
from sklearn.model_selection import GridSearchCV, KFold

from sklearn.neural_network import MLPRegressor
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

%matplotlib inline

In [4]:
Salary = pd.read_csv('./Kaggle_Salary.csv')

/home/hanwen/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,3,9,11,13,22,24,25,26,27,28,29,45,57,65,84,86,88,108,110,124,126,151,195,209,224,250,263,265,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,305,307,323,326,327,330,342,372,385,386,394,395) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# Drop three irrelavent columns
Salary.drop(['Unnamed: 0','index'], axis=1,inplace=True)

QuestionMapping = {}
for col in Salary.columns:
    QuestionMapping[col] = Salary.loc[0,col]
Salary.drop([0],inplace=True)
print(Salary.shape)
Salary.head()

# See how many null values are in each column
# Here we only deal with those colunmns that has null values more than zero but fewer that 1% of all data
less_001 = []
for col in Salary.columns:
    NullNum = Salary[col].isnull().sum()
    if (NullNum > 0) & (0.01 * Salary.shape[0] > NullNum):
        if ("Part" not in col) or (col in ['Q39_Part_1','Q39_Part_2']):
            less_001.append(col)
print("We drop nan rows in the below columns:",less_001)

for col in less_001:
    Salary.dropna(subset=[col],inplace=True)
print(Salary.shape)
between_001_020 = []
for col in Salary.columns:
    NullNum = Salary[col].isnull().sum()
    if (0.2 * Salary.shape[0] > NullNum) and (NullNum >= 0.01 * Salary.shape[0]):
        if ("Part" not in col) or (col in ['Q39_Part_1','Q39_Part_2']):
            between_001_020.append(col)
print("We use modes in these columns to replace nan value:",between_001_020)
for col in between_001_020:
    Salary[col].fillna(Salary[col].mode()[0],inplace=True)

between_020_050 = []
for col in Salary.columns:
    NullNum = Salary[col].isnull().sum()
    if (0.2 * Salary.shape[0]) < NullNum and (NullNum < 0.5 * Salary.shape[0]):
        if ("Part" not in col) or (col in ['Q39_Part_1','Q39_Part_2']):
            between_020_050.append(col)
print("We set nans in these columns as 'Unknown':",between_020_050)

for col in between_020_050:
    Salary.loc[Salary[col].isnull(),col] = 'Unknown'
more_050 = []
for col in Salary.columns:
    NullNum = Salary[col].isnull().sum()
    if 0.5 * Salary.shape[0] < NullNum:
        if ("Part" not in col) or (col in ['Q39_Part_1','Q39_Part_2']):
            more_050.append(col)
print("We drop these columns because large part of the data is missing:",more_050)

for col in more_050:
    Salary.drop([col], axis=1,inplace=True)
ALLCOL = Salary.columns.tolist()
def select_col(condition, allcol=ALLCOL):
    digit = len(condition)
    selected = []
    for i in allcol:
        if condition in i:
            if len(i)==digit:
                selected.append(i)
            elif i[digit] not in '0123456789':
                selected.append(i)                
    return selected
DummyCols = ['Q1','Q4','Q5','Q6','Q7','Q10','Q12_MULTIPLE_CHOICE',\
            'Q17','Q18','Q20','Q23','Q26','Q32','Q39_Part_1', 'Q39_Part_2',\
             'Q40','Q48']
for col in DummyCols:
    Salary = pd.get_dummies(data=Salary, columns=[col])
# print(Salary['Q3'].value_counts())
val_counts = Salary['Q3'].value_counts()
value_mask = Salary['Q3'].isin(val_counts.index[val_counts < 50])
Salary.loc[value_mask,'Q3'] = "Other"

Salary = pd.get_dummies(data=Salary, columns=['Q3'])
# print(Salary['Q22'].value_counts())
val_counts = Salary['Q22'].value_counts()
value_mask = Salary['Q22'].isin(val_counts.index[val_counts < 100])
Salary.loc[value_mask,'Q22'] = "Other"

Salary = pd.get_dummies(data=Salary, columns=['Q22'])
def Range_Normalize(value):
    if isinstance(value, str):
        if '-' in value:
            temp = value.split('-')
            return (float(temp[0])+float(temp[1]))/2
        if '+' in value:
            a = value.split('+')
            return float(a[0])
        else:
            return value
    else:
        return value
# Salary[select_col('Q2')]
Salary['Q2_NORMAL'] = Salary['Q2'].apply(Range_Normalize)
Salary.drop(['Q2'],axis=1,inplace=True)
# Salary[select_col('Q8')]
Salary['Q8_NORMAL'] = Salary['Q8'].apply(Range_Normalize)
Salary.drop(['Q8'],axis=1,inplace=True)
# Salary[select_col('Q24')]
def Q24Range_Normalize(value):
    mapdict = {'I have never written code and I do not want to learn':-1,
               'I have never written code but I want to learn':0,'< 1 year':0.5,
              '1-2 years':1.5,'3-5 years':4,'5-10 years':7.5,'10-20 years':15,
               '20-30 years':25,'30-40 years':35,'40+ years':40}
    return mapdict[value]
# Salary[select_col('Q2')]
Salary['Q24_NORMAL'] = Salary['Q24'].apply(Q24Range_Normalize)
Salary.drop(['Q24'],axis=1,inplace=True)
# Salary[select_col('Q25')]
Salary['Q25'].unique()
def Q25Range_Normalize(value):
    mapdict = {'I have never studied machine learning and I do not plan to':-1,
               'I have never studied machine learning but plan to learn in the future':0,
               '< 1 year':0.5,'1-2 years':1.5,'2-3 years':2.5,'3-4 years':3.5,'4-5 years':4.5,
               '5-10 years':7.5,'10-15 years':12.5,'20+ years':20}
    return mapdict[value]

Salary['Q25_NORMAL'] = Salary['Q25'].apply(Q25Range_Normalize)
Salary.drop(['Q25'],axis=1,inplace=True)
def Q41Range_Normalize(value):
    mapdict = {'Very important':10,
               'Slightly important':5,
               'No opinion; I do not know':2,
               'Unknown':0,
               'Not at all important':-10}
    return mapdict[value]

Salary.loc[Salary['Q41_Part_1'].isnull(),'Q41_Part_1'] = 'Unknown'
Salary.loc[Salary['Q41_Part_2'].isnull(),'Q41_Part_2'] = 'Unknown'
Salary.loc[Salary['Q41_Part_3'].isnull(),'Q41_Part_3'] = 'Unknown'

Salary['Q41_Part_1_NORMAL'] = Salary['Q41_Part_1'].apply(Q41Range_Normalize)
Salary.drop(['Q41_Part_1'],axis=1,inplace=True)
Salary['Q41_Part_2_NORMAL'] = Salary['Q41_Part_2'].apply(Q41Range_Normalize)
Salary.drop(['Q41_Part_2'],axis=1,inplace=True)
Salary['Q41_Part_3_NORMAL'] = Salary['Q41_Part_3'].apply(Q41Range_Normalize)
Salary.drop(['Q41_Part_3'],axis=1,inplace=True)
def Q43Range_Normalize(value):
    if isinstance(value, str):
        if '-' in value:
            temp = value.split('-')
            return (float(temp[0])+float(temp[1]))/2
        elif value == "0":
            return 0
        else:
            return float("nan")
    else:
        return value
Salary['Q43_NORMAL'] = Salary['Q43'].apply(Q43Range_Normalize)
Salary.drop(['Q43'],axis=1,inplace=True)
Salary['Q43_NORMAL'].fillna(Salary['Q43_NORMAL'].mode()[0],inplace=True)

Salary['Q46_NORMAL'] = Salary['Q46'].apply(Q43Range_Normalize)
Salary.drop(['Q46'],axis=1,inplace=True)
Salary['Q46_NORMAL'].fillna(Salary['Q46_NORMAL'].mode()[0],inplace=True)
# Salary[select_col('Q11')]
import math
def Binarize(value):
    if isinstance(value,str):
        return 1
    else:
        if value == 0 or math.isnan(value):
            return 0
        else:
            return 1
def binarize_col(columns, data=Salary):
    for col in columns:
        data[col] = data[col].apply(Binarize)
NormalCols = select_col('Q11_Part') + select_col('Q13_Part') + \
select_col('Q14_Part') + select_col('Q15_Part') + select_col('Q16_Part')\
+ select_col('Q19_Part') + select_col('Q21_Part') + select_col('Q27_Part')\
+ select_col('Q28_Part') + select_col('Q29_Part') + select_col('Q30_Part') +\
select_col('Q31_Part') + select_col('Q33_Part') + select_col('Q34_Part') +\
select_col('Q36_Part') + select_col('Q38_Part') + select_col('Q42_Part') + \
select_col('Q44_Part') + select_col('Q45_Part') + select_col('Q47_Part') + \
select_col('Q49_Part') + select_col('Q50_Part')
binarize_col(NormalCols)
import math
def TBinarize(value):
    if isinstance(value,str):
        return float(value)
    else:
        if math.isnan(value):
            return -1
        else:
            return value
def Tbinarize_col(columns, data=Salary):
    for col in columns:
        data[col] = data[col].apply(TBinarize)

Tbinarize_col(select_col('Q34_Part'))
Tbinarize_col(select_col('Q35_Part'))
for col in Salary.columns:
    if isinstance(Salary.loc[1,col],str):
        Salary[col] = Salary[col].apply(lambda x:float(x))
Salary.rename(columns={"Q9": "Yearly_compensation"},inplace=True)
Salary.rename(columns={"Time from Start to Finish (seconds)": "TimeUse"},inplace=True)
Salary.drop(Salary[Salary.TimeUse < 180].index,inplace=True)
for col in Salary.columns:
    if len(Salary[col].value_counts())==1:
        Salary.drop([col], axis=1,inplace=True)
Salary.drop(Salary[Salary.Yearly_compensation < 1000].index, inplace=True)
Salary.drop(Salary[Salary.Yearly_compensation > 350000].index, inplace=True)
X = Salary.drop(['Yearly_compensation'],axis=1)
Y = Salary['Yearly_compensation']

from sklearn.model_selection import train_test_split
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X,Y,test_size=0.3, random_state=20)
print(X.shape,X_Train.shape)

(15429, 395)
We drop nan rows in the below columns: ['Q5', 'Q8']
(15219, 395)
We use modes in these columns to replace nan value: ['Q10', 'Q12_MULTIPLE_CHOICE', 'Q18', 'Q23', 'Q24', 'Q25', 'Q26']
We set nans in these columns as 'Unknown': ['Q17', 'Q20', 'Q22', 'Q32', 'Q39_Part_1', 'Q39_Part_2', 'Q40', 'Q43', 'Q46', 'Q48']
We drop these columns because large part of the data is missing: ['Q37']
(13995, 605) (9796, 605)


In [6]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

def feature_scaling(X_Train, X_Test, method):
    methodmap = {'z-score':StandardScaler(),'min-max':MinMaxScaler(),'robust':RobustScaler()}
    if method not in ['z-score','min-max','robust']:
        print("We don't have this kind of method for normalization. Please choose from ['z-score','min-max','robust']")
        return pd.DataFrame(X_Train), pd.DataFrame(X_Test)
    else:
        ss = methodmap[method].fit(X_Train)
        X_Train = ss.transform(X_Train)
        X_Test = ss.transform(X_Test)
        return pd.DataFrame(X_Train), pd.DataFrame(X_Test)
X_Train_All, X_Test_All = feature_scaling(X_Train,X_Test,'z-score')

In [7]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.decomposition import PCA

def feature_elimination(X_Train, X_Test, Y_Train, method='pca', para=None):
    if method not in ['variance','pca','mul_info']:
        print("We don't have this kind of method for normalization. Please choose from ['variance','pca','mul_info']")
        return X_Train, X_Test
    else:
        if method=='variance':
            Para = para if para else (.008 * (1 - .008))
            sel = VarianceThreshold(threshold = Para).fit(X_Train)
            X_Train = sel.transform(X_Train)
            X_Test = sel.transform(X_Test)
            return pd.DataFrame(X_Train), pd.DataFrame(X_Test)
        elif method=='mul_info':
            Para = para if para else 200
            sel = SelectKBest(mutual_info_classif, k=Para).fit(X_Train,Y_Train)
            X_Train = sel.transform(X_Train)
            X_Test = sel.transform(X_Test)
            return pd.DataFrame(X_Train), pd.DataFrame(X_Test)
        else:
            Para = para if para else 200
            sel = PCA(n_components=Para).fit(X_Train)
            X_Train = sel.transform(X_Train)
            X_Test = sel.transform(X_Test)
            return pd.DataFrame(X_Train), pd.DataFrame(X_Test)

In [8]:
X_Train,X_Test = feature_elimination(X_Train_All,X_Test_All,Y_Train,'pca',400)

In [9]:
def bias(y_predict,y):
    y_predict = np.array(y_predict)
    y = np.array(y)
    avg = np.average(y_predict)
    return np.sqrt(np.average(np.power((avg-y),2)))
def variance(y_predict):
    y_predict = np.array(y_predict)
    return np.sqrt(np.var(y_predict))
from collections import OrderedDict

from sklearn.linear_model import LinearRegression as lr
from sklearn.linear_model import Lasso as lasso
from sklearn.linear_model import ElasticNet as elsnet
from sklearn.ensemble import RandomForestRegressor as rfr
from sklearn.ensemble import GradientBoostingRegressor as gbr

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

from sklearn.metrics import mean_squared_error, make_scorer, r2_score, mean_absolute_error
from sklearn.metrics import roc_auc_score, roc_curve, auc,confusion_matrix,accuracy_score

def two_score(y_true,y_pred):    
    mean_squared_error(y_true,y_pred) #set score here and not below if using MSE in GridCV
    score = r2_score(y_true,y_pred)
    return score

def run_kfold(model,X,y,folds=10):
    kf = KFold(n_splits=folds)
    kf.get_n_splits(X)
    score_keep = []
    trade_keep = []
    fold = 0
    print("The result with {}-fold cross-validation on training set".format(folds))

    for train_index, test_index in kf.split(X):
        fold += 1
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        clf = model.fit(X_train, y_train)
        y_train_predict = clf.predict(X_train)
        y_test_predict = clf.predict(X_test)

        train_r2 = r2_score(y_train, y_train_predict)
        train_rmse = np.sqrt(mean_squared_error(y_train, y_train_predict))
        train_mae = mean_absolute_error(y_test, y_test_predict)
        
        test_r2 = r2_score(y_test, y_test_predict)
        test_rmse = np.sqrt(mean_squared_error(y_test, y_test_predict))
        test_mae = mean_absolute_error(y_test, y_test_predict)
        # print("Fold {} R2 training score: {}, testing score: {}".format(fold,train_r2,test_r2))
        # print("RMSE training score: {}, testing score: {}".format(train_rmse,test_rmse))
        train_bia = bias(y_train_predict,y_train)
        train_var = variance(y_train_predict)
        train_total = train_bia + train_var
        
        test_bia = bias(y_test_predict,y_train)
        test_var = variance(y_test_predict)
        test_total = test_bia + test_var

        score_keep.append((train_r2, test_r2,train_rmse, test_rmse,train_mae,test_mae))
        trade_keep.append((train_bia,test_bia,train_var,test_var,train_total,test_total))
    score_df = pd.DataFrame(score_keep, columns=["train_R2","test_R2","train_RMSE","test_RMSE","train_MAE","test_MAE"])
    trade_df = pd.DataFrame(trade_keep, columns=["train_bia","test_bia","train_var","test_var","train_total","test_total"])
    print(score_df.describe()[1:4])
    return clf, score_df, trade_df

class Regressor(object):
    def __init__(self,train_data,train_targets):
        super().__init__()
        self.train_data = train_data
        self.train_targets = train_targets
        rfrhyperpara = {'n_estimators': list(range(90,120,5)),'max_depth': list(range(40,50,3)),'min_samples_leaf': [1, 2, 3]}
        gbrhyperpara = {'n_estimators': list(range(90,120,5)),'max_depth': list(range(4,15,3)),'min_samples_leaf': [1, 2]}
        self.models=OrderedDict([('LinearRegression',[lr(),None]),\
                            ('Lasso',[lasso(alpha=9.0,max_iter=2000,random_state=20),dict(alpha=np.arange(5,15,0.5))]),\
                            ('ElasticNet',[elsnet(l1_ratio=0.8),dict(l1_ratio=np.linspace(0,1,10))]),\
                            ('RandomForest',[rfr(n_estimators = 105, max_depth=40, min_samples_leaf=2),rfrhyperpara]),\
                            ('GradientBoosting',[gbr(),gbrhyperpara])])
        #1.1 and 2.70 # ElasticNet(alpha=1.0, l1_ratio=0.5）
    def train_model(self,model_name,scorer="R2",ifhyper_tune=False,hyper_paras="",folder=10):
        self.model_name = model_name
        model = self.models[model_name][0]
        if ifhyper_tune:
            if not hyper_paras:
                hyper_paras = self.models[model_name][1]
            # Grid search method for hyper-parameter tuning in which we use 5-fold cross validation
            if hyper_paras:
                if scorer=='R2':
                    acc_scorer = make_scorer(r2_score)
                elif scorer == 'RMSE':
                    acc_scorer = make_scorer(mean_squared_error)
                else:
                    acc_scorer = make_scorer(two_score, greater_is_better=True)
                grid = GridSearchCV(model, hyper_paras, cv=5, scoring=acc_scorer, n_jobs=-1) 

                grid.fit(self.train_data, self.train_targets)
                print('The best score of model {} through {}-cross validation is {}, with the best hyper-parameter {}\n'\
                    .format(model_name, 5, grid.best_score_, grid.best_params_))

                # Best model
                best_model = grid.best_estimator_
            else:
                best_model = model.fit(self.train_data,self.train_targets)
            train_pred = best_model.predict(self.train_data)
            R2 = r2_score(self.train_targets,train_pred)
            print('Through Hypertuning on {}, we get the best model and the results below:'.format(model_name))
            print('The R2 score is {}'.format(R2))
            RMSE = np.sqrt(mean_squared_error(self.train_targets,train_pred))
            print('The RMSE score is {}'.format(RMSE))
            MAE = mean_absolute_error(self.train_targets,train_pred)
            print('The MAE score is {}'.format(MAE))
            score_df = pd.DataFrame([(R2,RMSE,MAE)], columns=["R2", "RMSE","MAE"])
        else:
            best_model, score_df, trade_df = run_kfold(model,self.train_data,self.train_targets,folder)
        
        self.best_model = best_model
        if ifhyper_tune:
            return best_model, score_df
        else:
            return best_model, score_df, trade_df

    def test_model(self,test_data,test_targets,new_best=None):
        if new_best is not None:
            test_pred = new_best.predict(test_data)
        else:
            test_pred = self.best_model.predict(test_data)

        R2 = r2_score(test_targets, test_pred)
        RMSE = np.sqrt(mean_squared_error(test_targets,test_pred))
        MAE = mean_absolute_error(test_targets,test_pred)
        BIAS = bias(test_pred,test_targets)
        VAR = variance(test_pred)
        TOTAL = BIAS + VAR
        #test accuracy
        print("Result of Model {} is list below:".format(self.model_name))
        print('R2 score is {}'.format(R2))
        print('RMSE socre is {}'.format(RMSE))
        print('MeanAbsoluteError is {}'.format(MAE))
        print('Bias score is {}'.format(BIAS))
        print('Variance score is {}'.format(VAR))
        print('Bias+Variance score is {}'.format(TOTAL))
        return R2, RMSE, MAE, BIAS, VAR, TOTAL

In [10]:
PcaFeatureReg = Regressor(X_Train,Y_Train)

In [11]:
lr_model, lr_score, lr_trade = PcaFeatureReg.train_model('LinearRegression')

The result with 10-fold cross-validation on training set
      train_R2   test_R2    train_RMSE     test_RMSE     train_MAE  \
mean  0.592084  0.543050  33064.841036  34953.763895  23789.538421   
std   0.002802  0.026364    185.427076   1624.622823    639.548098   
min   0.586728  0.503513  32741.345410  32742.397357  22813.967068   

          test_MAE  
mean  23789.538421  
std     639.548098  
min   22813.967068  


In [12]:
lasso_model, lasso_score,lasso_trade = PcaFeatureReg.train_model('Lasso')

The result with 10-fold cross-validation on training set
      train_R2   test_R2    train_RMSE     test_RMSE     train_MAE  \
mean  0.592072  0.543340  33065.334139  34942.598017  23772.911417   
std   0.002802  0.026349    185.424382   1622.435139    637.484555   
min   0.586715  0.503710  32741.844199  32735.466885  22801.940146   

          test_MAE  
mean  23772.911417  
std     637.484555  
min   22801.940146  


In [13]:
elc_model, elc_score, elc_trade = PcaFeatureReg.train_model('ElasticNet')

The result with 10-fold cross-validation on training set
      train_R2   test_R2    train_RMSE     test_RMSE     train_MAE  \
mean  0.586864  0.546145  33275.717746  34836.760560  23461.936906   
std   0.002809  0.025860    184.489814   1644.817749    639.223899   
min   0.581495  0.506851  32953.514032  32629.243779  22557.429044   

          test_MAE  
mean  23461.936906  
std     639.223899  
min   22557.429044  


In [14]:
rfr_model, rfr_score, rfr_trade = PcaFeatureReg.train_model('RandomForest')

The result with 10-fold cross-validation on training set
      train_R2   test_R2    train_RMSE     test_RMSE     train_MAE  \
mean  0.881849  0.320334  17794.885894  42636.247967  30313.752273   
std   0.001398  0.022700    128.888503   1604.222084    576.410245   
min   0.879750  0.280440  17545.751662  41312.676770  29345.907957   

          test_MAE  
mean  30313.752273  
std     576.410245  
min   29345.907957  


In [15]:
gdb_model, gdb_score, gdb_trade = PcaFeatureReg.train_model('GradientBoosting')

The result with 10-fold cross-validation on training set


KeyboardInterrupt: 

In [ ]:
all_score = [lr_score,lasso_score,elc_score,rfr_score,gdb_score]
xlabels = ['LinearRegression','Lasso','ElasticNet','RandomForest','GradientBoosting']
ylabels = all_score[0].columns
for i in range(len(ylabels)):
    if not i%2:
        continue
    score = [j.iloc[:,i] for j in all_score]
    plt.figure()
    plt.ylabel(ylabels[i])
    plt.boxplot(score, labels=xlabels)
    plt.xticks(rotation=30)
    plt.show()
all_trade = [lr_trade,lasso_trade,elc_trade,rfr_trade,gdb_trade]
ylabels = all_trade[0].columns
for i in range(len(ylabels)):
    if not i%2:
        continue
    score = [j.iloc[:,i] for j in all_trade]
    plt.figure()
    plt.ylabel(ylabels[i])
    plt.boxplot(score, labels=xlabels)
    plt.xticks(rotation=30)
    plt.show()